In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e3/sample_submission.csv
/kaggle/input/playground-series-s5e3/train.csv
/kaggle/input/playground-series-s5e3/test.csv


In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from io import StringIO
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import plotly.express as px
import matplotlib as mpl
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import openpyxl
import plotly.graph_objects as go
from matplotlib.sankey import Sankey
from xgboost import XGBRegressor
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import StackingRegressor

In [3]:
rain_train = "/kaggle/input/playground-series-s5e3/train.csv"
rain_test = "/kaggle/input/playground-series-s5e3/test.csv"

In [4]:
rain_train_df = pd.read_csv(rain_train)

In [5]:
rain_test_df = pd.read_csv(rain_test)

In [6]:
print (rain_train_df['humidity'])

0       87.0
1       95.0
2       75.0
3       95.0
4       52.0
        ... 
2185    97.0
2186    91.0
2187    79.0
2188    92.0
2189    89.0
Name: humidity, Length: 2190, dtype: float64


# Outline

Objective: Create a deep learning model to predict rainfall via machine learning.

How: Model building. 

Evaluation: Submissions are evaluated on area under the ROC curve between the predicted probability and the observed target. 

Prediction Target: rainfall column in rain_train. 

In [7]:
def categorize_wind_dir(degree):
    """
    Categorizes wind direction degrees into cardinal directions.
    Assumes degree is between 0° and 360°.
    """
    if 45 <= degree < 135:
        return 'East'
    elif 135 <= degree < 225:
        return 'South'
    elif 225 <= degree < 315:
        return 'West'
    else:
        return 'North'  # Includes 315–360 and 0–44

In [8]:
wind_source_map = {'North': 0, 'East': 1, 'South': 2, 'West': 3}

In [9]:
for df in [rain_train_df, rain_test_df]:
    df['cloud_pressure_temperature'] = df['cloud'] * df['pressure']
    df['wind_source'] = df['winddirection'].apply(categorize_wind_dir)
    df['wind_source'] = df['wind_source'].map(wind_source_map)

X = rain_train_df[["temparature", "maxtemp", "mintemp", "dewpoint", "pressure", "windspeed"]]
y = rain_train_df["rainfall"]

Add a constant term for the intercept
X = sm.add_constant(X)

Fit linear regression
model = sm.OLS(y, X).fit()
print(model.summary())

Plot residuals
residuals = model.resid
sns.histplot(residuals, kde=True)
plt.title("Residual Distribution")
plt.show()

p_values = model.pvalues

Find significant variables (p-value < 0.05)
significant_vars = p_values[p_values < 0.05].index.tolist()

Highlight significant variables
print("\n🚀 **Significant Variables at α = 0.05:**")
for var in significant_vars:
    print(f"✅ {var} (p-value: {p_values[var]:.4f})") e

# Modelling

In [10]:
feature_columns = ['dewpoint', 'humidity', 'sunshine', 'temparature', 'wind_source', 'cloud_pressure_temperature']

In [11]:
# Split training data
X_train = rain_train_df[feature_columns]
y_train = rain_train_df['rainfall']

In [12]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [13]:
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split

try:
    from catboost import CatBoostRegressor
    cat_model = CatBoostRegressor(verbose=0, random_state=42)
except ImportError:
    print("CatBoost not installed, using only RF and KNN")
    cat_model = None

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
knn_model = KNeighborsRegressor(n_neighbors=5)

base_models = [('rf', rf_model), ('knn', knn_model)]
if cat_model:
    base_models.append(('cat', cat_model))

# Stack models
stack_model = StackingRegressor(
    estimators=base_models,
    final_estimator=Ridge()
)

# Fit and predict
stack_model.fit(X_train, y_train)

StackingRegressor(estimators=[('rf', RandomForestRegressor(random_state=42)),
                              ('knn', KNeighborsRegressor()),
                              ('cat',
                               <catboost.core.CatBoostRegressor object at 0x7f4f6f204100>)],
                  final_estimator=Ridge())

In [14]:
X_test = rain_test_df[feature_columns]
rain_test_df['rainfall'] = stack_model.predict(X_test)

In [15]:
submission = rain_test_df[['id', 'rainfall']]

# If ordered and un_ordered are required and available in test:
# submission['ordered'] = pastry_test_df['ordered']
# submission['un_ordered'] = pastry_test_df['un_ordered']

# Save submission file
submission.to_csv('submission.csv', index=False)
print("Submission file saved successfully!")

Submission file saved successfully!
